# SetUP

In [ ]:
!pip install requests
!pip install bs4

In [ ]:
import requests
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib

In [ ]:
key = "J%2FMF%2FFZu79F5m8IclqQvnPX3FEnkqzmgeDwH38xvKaB49HL3PrZ91d5TE0qApitoEBHy%2F5wfhDKhDXJlYVOlQA%3D%3D"

# 1) 국문 관광정보 서비스 API

Request 파라미터

In [ ]:
"""
#arrange: (A=제목순, C=수정일순 , D=생성일순) 대표이미지가반드시있는정렬 (O=제목순, Q=수정일순, R=생성일순)

params = {
    "arrange": , #정렬구분
    "contentTypeid": , #콘텐츠타입구분
    "areaCode": , #지역구분
    "sigunguCode": , #시군구구분 (areaCode 반드시 필요)
    "cat1": , #대분류1
    "cat2": , #대분류2
    "cat3": , #대분류3
    "modifiedtime":  ,#콘텐츠수정일
    "mapX": ,#WGS84 GPS 경도
    "mapY: ,#WGS84 GPS 위도
    "radius": ,#거리반경(m단위)
    }
"""

1-1) areaCode1 : 지역코드조회

In [ ]:
# 지역 별 code 조회 기능
service = "areaCode1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=TestApp&_type=json"

res = requests.get(url)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['rnum', 'code', 'name'], dtype='object')


,rnum,code,name
0,1,1,서울
1,2,2,인천
2,3,3,대전
3,4,4,대구
4,5,5,광주


1-2) categoryCode1 : 서비스분류코드조회

In [ ]:
# 장소 분류 카테고리 code 조회 기능
service = "categoryCode1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=TestApp&_type=json"

res = requests.get(url)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['code', 'name', 'rnum'], dtype='object')


,code,name,rnum
0,A01,자연,1
1,A02,인문(문화/예술/역사),2
2,A03,레포츠,3
3,A04,쇼핑,4
4,A05,음식,5


1-3) areaBasedList1 : 지역기반관광정보조회

In [ ]:
# 장소별 종합정보 조회 기능(위치, 상호, 위경도 좌표, 카테고리 등)
service = "areaBasedList1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=TestApp&_type=json"

res = requests.get(url)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['addr1', 'addr2', 'areacode', 'booktour', 'cat1', 'cat2', 'cat3',
       'contentid', 'contenttypeid', 'createdtime', 'firstimage',
       'firstimage2', 'cpyrhtDivCd', 'mapx', 'mapy', 'mlevel', 'modifiedtime',
       'sigungucode', 'tel', 'title', 'zipcode'],
      dtype='object')


,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,...,firstimage2,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode
0,충청남도 공주시 감영길 3,(반죽동),34,,A04,A0401,A04010600,2750144,38,20210928012320,...,,,127.1216721795,36.4529297875,6,20231206145555,1,,가가상점,32546
1,부산광역시 부산진구 중앙번영로 (6),,6,,A05,A0502,A05020300,2805408,39,20220125140006,...,,,129.0598281416,35.1448074253,6,20240104133528,7,,가가와,47361
2,충청남도 공주시 당간지주길 10,(반죽동),34,,A02,A0206,A02061000,2750143,38,20210928012011,...,,,127.1219610659,36.4520687798,6,20231103165421,1,,가가책방,32549
3,전라남도 신안군 흑산면 가거도길 38-2,(흑산면),38,0,A01,A0101,A01011300,127480,12,20030905090000,...,,,125.1125145360,34.0740166502,6,20240514161814,12,,가거도(소흑산도),58866
4,충청북도 청주시 흥덕구 가경로149번길 24-2,,33,,A04,A0401,A04010200,1433504,38,20111111014944,...,,,127.4358669978,36.6278652990,6,20230407105028,10,,가경 터미널시장,28393


1-4) locationBasedList1 : 위치기반관광정보조회

In [ ]:
# 위/경도 좌표 및 거리반경 기반 관광정보 조회 / 예외처리 필요(반경 내 존재X 가능)
service = "locationBasedList1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=TestApp&_type=json"

params = {
    "mapX":127.12,
    "mapY":36.45,
    "radius":30000
}

res = requests.get(url,params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['addr1', 'addr2', 'areacode', 'booktour', 'cat1', 'cat2', 'cat3',
       'contentid', 'contenttypeid', 'createdtime', 'dist', 'firstimage',
       'firstimage2', 'cpyrhtDivCd', 'mapx', 'mapy', 'mlevel', 'modifiedtime',
       'sigungucode', 'tel', 'title'],
      dtype='object')


,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,...,firstimage,firstimage2,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title
0,충청남도 공주시 효심1길 12-1,,34,,A05,A0502,A05020900,2749728,39,20210927202213,...,,,,127.1195363470,36.4496967877,6,20230629092950,1,,곡물집
1,충청남도 공주시 큰샘3길 8,,34,,B02,B0201,B02011600,2708325,32,20210308235736,...,http://tong.visitkorea.or.kr/cms/resource/81/2...,http://tong.visitkorea.or.kr/cms/resource/81/2...,Type3,127.1200307170,36.4508188891,,20230410103446,1,041-960-5525,봉황재 한옥 게스트하우스 [한국관광 품질인증/Korea Quality]
2,충청남도 공주시 봉황로 54-1,,34,,A04,A0401,A04010600,2751008,38,20210929021403,...,,,,127.1210445097,36.4510105524,6,20231109105708,1,,고마다락
3,충청남도 공주시 봉황산1길 1-2,(반죽동),34,,A05,A0502,A05020100,2736661,39,20210826191033,...,http://tong.visitkorea.or.kr/cms/resource/39/2...,http://tong.visitkorea.or.kr/cms/resource/39/2...,Type3,127.1206250807,36.4518298957,6,20230802201507,1,,곰골식당
4,충청남도 공주시 제민1길 18,(봉황동),34,0,A02,A0201,A02010900,2613458,12,20190729224016,...,http://tong.visitkorea.or.kr/cms/resource/07/2...,http://tong.visitkorea.or.kr/cms/resource/07/2...,Type3,127.1225249329,36.4509911363,6,20231013144836,1,,공주제일교회


1-5) searchKeyword1 : 키워드검색조회

In [ ]:
# 키워드 기반 관광정보 조회 / 예외처리 필요(존재X 가능), 키워드: 인코딩 없이 한글로 기입(장소 키워드인듯)
service = "searchKeyword1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

word = "청주"

params = {
    "keyword":word,
}

res = requests.get(url,params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['addr1', 'addr2', 'areacode', 'booktour', 'cat1', 'cat2', 'cat3',
       'contentid', 'contenttypeid', 'createdtime', 'firstimage',
       'firstimage2', 'cpyrhtDivCd', 'mapx', 'mapy', 'mlevel', 'modifiedtime',
       'sigungucode', 'tel', 'title'],
      dtype='object')


,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,firstimage,firstimage2,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title
0,충청북도 청주시 상당구 단재로 557,1층,33,,A05,A0502,A05020100,2860359,39,20220927094534,http://tong.visitkorea.or.kr/cms/resource/50/2...,http://tong.visitkorea.or.kr/cms/resource/50/2...,Type3,127.5106504544,36.5816858255,6,20240416171904,10,,고화갈비살 청주본점
1,충청북도 청주시 상당구 명암로 143,,33,0,A02,A0206,A02060100,129779,14,20071106102944,,,,127.5127178664,36.6485420822,6,20240130120421,10,,국립청주박물관
2,충청북도 청주시 상당구 명암로 143,(명암동),33,0,A02,A0206,A02060100,130143,14,20071106104408,http://tong.visitkorea.or.kr/cms/resource/51/1...,http://tong.visitkorea.or.kr/cms/resource/51/1...,Type3,127.5119706779,36.6497224889,6,20231115164526,10,,국립청주박물관 어린이박물관
3,충청북도 청주시 청원구 상당로 314,(내덕동),33,0,A02,A0206,A02060500,2598367,14,20190420021156,http://tong.visitkorea.or.kr/cms/resource/49/2...,http://tong.visitkorea.or.kr/cms/resource/49/2...,Type3,127.4902363546,36.6555769823,6,20230927114218,10,,국립현대미술관 청주
4,충청북도 청주시 청원구 충청대로 114,,33,,B02,B0201,B02010100,1992755,32,20150324182410,http://tong.visitkorea.or.kr/cms/resource/04/2...,http://tong.visitkorea.or.kr/cms/resource/04/2...,Type3,127.4945134437,36.6666091257,6,20240307174352,10,043-290-1000,그랜드플라자 청주호텔


1-6) searchFestival1 : 행사정보조회

In [ ]:
# 날짜 기반 행사정보 조회 / 예외처리 필요(존재X 가능)
service = "searchFestival1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

date = "20240605" #YYYYMMDD 형태

params = {
    "eventStartDate":date
}

res = requests.get(url,params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['addr1', 'addr2', 'booktour', 'cat1', 'cat2', 'cat3', 'contentid',
       'contenttypeid', 'createdtime', 'eventstartdate', 'eventenddate',
       'firstimage', 'firstimage2', 'cpyrhtDivCd', 'mapx', 'mapy', 'mlevel',
       'modifiedtime', 'areacode', 'sigungucode', 'tel', 'title'],
      dtype='object')


,addr1,addr2,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,eventstartdate,...,firstimage2,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,areacode,sigungucode,tel,title
0,강원특별자치도 강릉시 단오장길 1,,,A02,A0207,A02070200,531391,15,20080415004012,20240606,...,http://tong.visitkorea.or.kr/cms/resource/01/3...,Type3,128.8950169869,37.7482134974,6,20240328141214,32,1,033-641-1593,강릉단오제
1,전라남도 강진군 강진읍 고성길 174,,,A02,A0207,A02070200,3304617,15,20240603135727,20240628,...,http://tong.visitkorea.or.kr/cms/resource/16/3...,Type3,126.7624529636,34.6522557433,6,20240603135847,38,1,061-430-3354,강진수국길축제
2,경상남도 거제시 옥포로2길 14,,,A02,A0207,A02070200,1327778,15,20110708235526,20240601,...,http://tong.visitkorea.or.kr/cms/resource/47/2...,Type3,128.6966193745,34.8905347934,6,20240527154135,36,1,055-680-1000,거제 옥포대첩 축제
3,경상남도 거창군 거창읍 수남로 2181,,,A02,A0208,A02080200,142069,15,20070709090000,20240726,...,http://tong.visitkorea.or.kr/cms/resource/78/3...,Type3,127.9108367626,35.6737770003,6,20240522140439,36,2,055-945-8455~6,거창국제연극제
4,서울특별시 종로구 사직로 161 경복궁,,,A02,A0207,A02070200,2615461,15,20190816224954,20240417,...,http://tong.visitkorea.or.kr/cms/resource/99/2...,Type3,126.9770319156,37.5788944508,6,20240514170158,1,23,1522-2295,경복궁 생과방


1-7) searchStay1 : 숙박정보조회

In [ ]:
# 숙박정보목록 조회
service = "searchStay1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

res = requests.get(url)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['addr1', 'addr2', 'areacode', 'benikia', 'cat1', 'cat2', 'cat3',
       'contentid', 'contenttypeid', 'createdtime', 'firstimage',
       'firstimage2', 'cpyrhtDivCd', 'goodstay', 'hanok', 'mapx', 'mapy',
       'mlevel', 'modifiedtime', 'tel', 'title', 'booktour', 'sigungucode'],
      dtype='object')


,addr1,addr2,areacode,benikia,cat1,cat2,cat3,contentid,contenttypeid,createdtime,...,goodstay,hanok,mapx,mapy,mlevel,modifiedtime,tel,title,booktour,sigungucode
0,경상북도 안동시 하회남촌길 69-5,,35,,B02,B0201,B02011600,2465071,32,20161220190700,...,,,128.5175868107,36.5376537450,6,20230413082505,054-855-8552,가경재 [한국관광 품질인증/Korea Quality],,11
1,서울특별시 송파구 송파대로28길 5,(가락동),1,,B02,B0201,B02010100,142785,32,20040426090000,...,,,127.1166298703,37.4966565128,6,20231227134640,02-400-6641~3,가락관광호텔,,18
2,전북특별자치도 전주시 완산구 한지길 68,(풍남동3가),37,,B02,B0201,B02011600,2671267,32,20200908224549,...,,,127.1535667664,35.8173223769,,20240118105019,,가락청,,12
3,경기도 파주시 소라지로327번길 126-21,(송촌동),31,,B02,B0201,B02011600,2627867,32,20191021220426,...,,,126.6891949040,37.7547493903,,20201224014445,,가람나무,,27
4,경상북도 안동시 풍천면 하회종가길 76-6,(풍천면),35,,B02,B0201,B02011600,1865597,32,20131202181422,...,,,128.5163479914,36.5390674494,6,20230901161045,010-3849-7542,가람초연재[한국관광 품질인증/Korea Quality],,11


1-8) detailCommon1 : 공통정보조회(상세정보1)

In [ ]:
# 공통정보조회(상세정보1) : contentId 필수
service = "detailCommon1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

params = {
    "contentId":"126508",
    "defaultYN":"Y", #기본정보조회
    "firstImageYN":"Y", #대표이미지조회
    "areacodeYN":"Y", #지역코드조회
    "catcodeYN":"Y", #카테고리조회
    "addrinfoYN":"Y", #주소조회
    "mapinfoYN":"Y", #좌표X,Y조회
    "overviewYN":"Y" #콘텐츠개요조회
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['contentid', 'contenttypeid', 'title', 'createdtime', 'modifiedtime',
       'tel', 'telname', 'homepage', 'booktour', 'firstimage', 'firstimage2',
       'cpyrhtDivCd', 'areacode', 'sigungucode', 'cat1', 'cat2', 'cat3',
       'addr1', 'addr2', 'zipcode', 'mapx', 'mapy', 'mlevel', 'overview'],
      dtype='object')


,contentid,contenttypeid,title,createdtime,modifiedtime,tel,telname,homepage,booktour,firstimage,...,cat1,cat2,cat3,addr1,addr2,zipcode,mapx,mapy,mlevel,overview
0,126508,12,경복궁,20041230090000,20240605084936,,,"<a href=""https://royal.khs.go.kr/ROYAL/content...",1,http://tong.visitkorea.or.kr/cms/resource/33/2...,...,A02,A0201,A02010100,서울특별시 종로구 사직로 161,,03045,126.9769930325,37.5788222356,6,경복궁은 1395년 태조 이성계에 의해서 새로운 조선왕조의 법궁으로 지어졌다. 경복...


1-9) detailIntro1 : 소개정보조회(상세정보2)

In [ ]:
# 소개정보조회(상세정보2) : contentId, contentTypeId 필수 / contentType 따라 response 값들이 달라짐.
service = "detailIntro1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

params = {
    "contentId":"2674675",
    "contentTypeId":"15"
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['contentid', 'contenttypeid', 'sponsor1', 'sponsor1tel', 'sponsor2',
       'sponsor2tel', 'eventenddate', 'playtime', 'eventplace',
       'eventhomepage', 'agelimit', 'bookingplace', 'placeinfo', 'subevent',
       'program', 'eventstartdate', 'usetimefestival', 'discountinfofestival',
       'spendtimefestival', 'festivalgrade'],
      dtype='object')


,contentid,contenttypeid,sponsor1,sponsor1tel,sponsor2,sponsor2tel,eventenddate,playtime,eventplace,eventhomepage,agelimit,bookingplace,placeinfo,subevent,program,eventstartdate,usetimefestival,discountinfofestival,spendtimefestival,festivalgrade
0,2674675,15,수원문화재단,031-290-3563,"경기관광공사, 수원문화재단",,20241231,연중(밤 10시 이후 제한),수원화성 일원,,,,,,,20220101,"7,500원",,,


1-10) detailInfo1 : 반복정보조회(상세정보3)

In [ ]:
# 반복정보조회(상세정보3) : contentId, contentTypeId 필수 / 숙박=객실정보, 여행코스=코스정보 제공 / contentTypeId 따라 response 달라짐
service = "detailInfo1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

params = {
    "contentId":"2674675",
    "contentTypeId":"15"
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['contentid', 'contenttypeid', 'serialnum', 'infoname', 'infotext',
       'fldgubun'],
      dtype='object')


,contentid,contenttypeid,serialnum,infoname,infotext,fldgubun
0,2674675,15,0,행사소개,스마트폰으로 수원화성 일원에서 진행하는 방탈출 게임 형식의 콘텐츠로 증강현실 등 다...,2


1-11) detailImage1 : 이미지정보조회(상세정보4)

In [ ]:
# 이미지정보조회(상세정보4) : imageYN, subImageYN로 이미지 정보 받기 / cpyrthDivCd: Type1(저작권 제1유형, 출처표시권장), Type3(1유형+변경금지)
# originalimgurl : 원본이미지, smallimageurl : 썸네일 이미지
service = "detailImage1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

params = {
    "contentId":"1095732",
    "imageYN":"Y",
    "subImageYN":"Y"
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['contentid', 'originimgurl', 'imgname', 'smallimageurl', 'cpyrhtDivCd',
       'serialnum'],
      dtype='object')


,contentid,originimgurl,imgname,smallimageurl,cpyrhtDivCd,serialnum
0,1095732,http://tong.visitkorea.or.kr/cms/resource/67/3...,서울빛초롱축제 포스터,http://tong.visitkorea.or.kr/cms/resource/67/3...,Type3,3035567_3
1,1095732,http://tong.visitkorea.or.kr/cms/resource/68/3...,서울빛초롱축제 1,http://tong.visitkorea.or.kr/cms/resource/68/3...,Type3,3035568_2
2,1095732,http://tong.visitkorea.or.kr/cms/resource/69/3...,서울빛초롱축제 2,http://tong.visitkorea.or.kr/cms/resource/69/3...,Type3,3035569_1
3,1095732,http://tong.visitkorea.or.kr/cms/resource/81/3...,서울빛초롱축제 3,http://tong.visitkorea.or.kr/cms/resource/81/3...,Type3,3037181_4
4,1095732,http://tong.visitkorea.or.kr/cms/resource/82/3...,서울빛초롱축제 4,http://tong.visitkorea.or.kr/cms/resource/82/3...,Type3,3037182_5


1-12) areaBasedSyncList1 : 국문관광정보 동기화목록조회

In [ ]:
# 국문관광정보 동기화목록조회 / showflag : 컨텐츠 표출 여부, 0 or 1을 parameter로 넣어 해당 값과 동일한 결과만 받을 수 있음 << 종합적인 정보 다 때려넣은듯?

service = "areaBasedSyncList1"
url = f"http://apis.data.go.kr/B551011/KorService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['addr1', 'addr2', 'areacode', 'booktour', 'cat1', 'cat2', 'cat3',
       'contentid', 'contenttypeid', 'createdtime', 'firstimage',
       'firstimage2', 'cpyrhtDivCd', 'mapx', 'mapy', 'mlevel', 'modifiedtime',
       'sigungucode', 'tel', 'title', 'zipcode', 'showflag'],
      dtype='object')


,addr1,addr2,areacode,booktour,cat1,cat2,cat3,contentid,contenttypeid,createdtime,...,cpyrhtDivCd,mapx,mapy,mlevel,modifiedtime,sigungucode,tel,title,zipcode,showflag
0,충청남도 공주시 감영길 3,(반죽동),34,,A04,A0401,A04010600,2750144,38,20210928012320,...,,127.1216721795,36.4529297875,6,20231206145555,1,,가가상점,32546,1
1,부산광역시 부산진구 중앙번영로 (6),,6,,A05,A0502,A05020300,2805408,39,20220125140006,...,,129.0598281416,35.1448074253,6,20240104133528,7,,가가와,47361,1
2,충청남도 공주시 당간지주길 10,(반죽동),34,,A02,A0206,A02061000,2750143,38,20210928012011,...,,127.1219610659,36.4520687798,6,20231103165421,1,,가가책방,32549,1
3,전라남도 신안군 흑산면 가거도길 38-2,(흑산면),38,0,A01,A0101,A01011300,127480,12,20030905090000,...,,125.1125145360,34.0740166502,6,20240514161814,12,,가거도(소흑산도),58866,1
4,충청북도 청주시 흥덕구 가경로149번길 24-2,,33,,A04,A0401,A04010200,1433504,38,20111111014944,...,,127.4358669978,36.6278652990,6,20230407105028,10,,가경 터미널시장,28393,1


# 2) 관광빅데이터 API

2-1) metcoRegnVisitrDDList : 광역지자체 방문자수

In [ ]:
# 관광데이터(방문자수) 조회
# startYmd:조회 시작연월일, endYmd:조회 끝연월일은 필수
service = "metcoRegnVisitrDDList"
url = f"http://apis.data.go.kr/B551011/DataLabService/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

params = {
    "startYmd":"20210513",
    "endYmd":"20210516"
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['areaCode', 'areaNm', 'daywkDivCd', 'daywkDivNm', 'touDivCd',
       'touDivNm', 'touNum', 'baseYmd'],
      dtype='object')


,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
0,11,서울특별시,4,목요일,1,현지인(a),4828435.5,20210513
1,11,서울특별시,4,목요일,2,외지인(b),1255828.0,20210513
2,11,서울특별시,4,목요일,3,외국인(c),879.3299999999997,20210513
3,11,서울특별시,5,금요일,1,현지인(a),4865997.5,20210514
4,11,서울특별시,5,금요일,2,외지인(b),1378545.0,20210514


2-2) locgoRegnVisitrDDList : 기초지자체 방문자수

In [ ]:
# 관광데이터(방문자수) 조회
# startYmd:조회 시작연월일, endYmd:조회 끝연월일은 필수
service = "locgoRegnVisitrDDList"
url = f"http://apis.data.go.kr/B551011/DataLabService/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

params = {
    "startYmd":"20210513",
    "endYmd":"20210516"
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['signguCode', 'signguNm', 'daywkDivCd', 'daywkDivNm', 'touDivCd',
       'touDivNm', 'touNum', 'baseYmd'],
      dtype='object')


,signguCode,signguNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
0,11110,종로구,4,목요일,1,현지인(a),176473.5,20210513
1,11110,종로구,4,목요일,2,외지인(b),317425.5,20210513
2,11110,종로구,4,목요일,3,외국인(c),43.38,20210513
3,11140,중구,4,목요일,1,현지인(a),179354.0,20210513
4,11140,중구,4,목요일,2,외지인(b),322980.0,20210513


# 3) 생태관광 정보활용 API

3-1) areaCode1 : 지역코드조회

In [ ]:
# 관광데이터(방문자수) 조회
# startYmd:조회 시작연월일, endYmd:조회 끝연월일은 필수

# *** 1)에서와 3)에서 지역코드는 완전히 동일함. (검증완료)

service = "areaCode1"
url = f"http://apis.data.go.kr/B551011/GreenTourService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

res = requests.get(url)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['code', 'name', 'rnum'], dtype='object')


,code,name,rnum
0,1,서울,1
1,2,인천,2
2,3,대전,3
3,4,대구,4
4,5,광주,5


3-2) areaBasedList1 : 지역기반생태관광정보조회

In [ ]:
# 지역기반생태관광정보 / areaCode, sigunguCode 파라미터 사용 가능
service = "areaBasedList1"
url = f"http://apis.data.go.kr/B551011/GreenTourService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

res = requests.get(url)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['addr', 'areacode', 'contentid', 'createdtime', 'mainimage',
       'cpyrhtDivCd', 'modifiedtime', 'sigungucode', 'subtitle', 'summary',
       'tel', 'telname', 'title'],
      dtype='object')


,addr,areacode,contentid,createdtime,mainimage,cpyrhtDivCd,modifiedtime,sigungucode,subtitle,summary,tel,telname,title
0,전라남도 순천시 국가정원1호길 47,38,2549246,20180526011631,http://tong.visitkorea.or.kr/cms/resource/24/2...,Type3,20240605160156,11,,순천만을 보호하기 위하여 조성한 순천만국가정원은 순천 도사동 일대 정원부지 112만...,061-749-2728,순천만국가정원,순천만국가정원
1,충청북도 보은군 속리산면 법주사로 84,33,2547328,20180513203608,http://tong.visitkorea.or.kr/cms/resource/95/2...,Type3,20240605113943,3,,1970년 6번째 국립공원으로 지정되었으며 예로부터 제2금강 또는 소금강이라 불릴 ...,043-542-5267,속리산국립공원관리사무소,속리산국립공원
2,강원특별자치도 고성군 현내면 금강산로 481,32,2508605,20180612190023,http://tong.visitkorea.or.kr/cms/resource/38/2...,Type3,20240604151251,2,,우리나라 최북단 강원도 고성은 남한 고성과 북한 고성으로 나뉜 분단군이다. 이런 지...,고성군청 관광문화체육과 033-680-3365,통일전망대 033-682-0088,고성 DMZ
3,대전광역시 동구 천개동로 41,3,2547521,20180513232046,http://tong.visitkorea.or.kr/cms/resource/07/2...,Type3,20240604112018,2,,상수원보호지역인 청정한 대청호숫가에 위치하여 자연과 사람이 조화를 이루는 환경 속에...,042-251-4781,대청호자연생태관,대청호자연생태관
4,강원특별자치도 양구군 해안면 해안서화로 35,32,2508606,20170923194609,http://tong.visitkorea.or.kr/cms/resource/19/2...,Type3,20240604110544,6,,강원도 양구DMZ는 군사 통제구역으로 잘 보존된 자연경관을 둘러볼 수 있는 생태관광...,033-480-7251,양구군청 문화관광과,양구DMZ


3-3) areaBasedSyncList1 : 생태관광정보동기화조회

In [ ]:
# 생태관광정보동기화조회 / showflag(표출여부), modifiedtime(수정일), areaCode(지역코드), sigunguCode(시군구코드) 파라미터 사용 가능
service = "areaBasedSyncList1"
url = f"http://apis.data.go.kr/B551011/GreenTourService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=10&_type=json"

res = requests.get(url)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['addr', 'areacode', 'contentid', 'createdtime', 'mainimage',
       'cpyrhtDivCd', 'modifiedtime', 'sigungucode', 'subtitle', 'summary',
       'tel', 'telname', 'title', 'showflag'],
      dtype='object')


,addr,areacode,contentid,createdtime,mainimage,cpyrhtDivCd,modifiedtime,sigungucode,subtitle,summary,tel,telname,title,showflag
0,전라남도 순천시 국가정원1호길 47,38,2549246,20180526011631,http://tong.visitkorea.or.kr/cms/resource/24/2...,Type3,20240605160156,11,,순천만을 보호하기 위하여 조성한 순천만국가정원은 순천 도사동 일대 정원부지 112만...,061-749-2728,순천만국가정원,순천만국가정원,1
1,충청북도 보은군 속리산면 법주사로 84,33,2547328,20180513203608,http://tong.visitkorea.or.kr/cms/resource/95/2...,Type3,20240605113943,3,,1970년 6번째 국립공원으로 지정되었으며 예로부터 제2금강 또는 소금강이라 불릴 ...,043-542-5267,속리산국립공원관리사무소,속리산국립공원,1
2,강원특별자치도 고성군 현내면 금강산로 481,32,2508605,20180612190023,http://tong.visitkorea.or.kr/cms/resource/38/2...,Type3,20240604151251,2,,우리나라 최북단 강원도 고성은 남한 고성과 북한 고성으로 나뉜 분단군이다. 이런 지...,고성군청 관광문화체육과 033-680-3365,통일전망대 033-682-0088,고성 DMZ,1
3,대전광역시 동구 천개동로 41,3,2547521,20180513232046,http://tong.visitkorea.or.kr/cms/resource/07/2...,Type3,20240604112018,2,,상수원보호지역인 청정한 대청호숫가에 위치하여 자연과 사람이 조화를 이루는 환경 속에...,042-251-4781,대청호자연생태관,대청호자연생태관,1
4,강원특별자치도 양구군 해안면 해안서화로 35,32,2508606,20170923194609,http://tong.visitkorea.or.kr/cms/resource/19/2...,Type3,20240604110544,6,,강원도 양구DMZ는 군사 통제구역으로 잘 보존된 자연경관을 둘러볼 수 있는 생태관광...,033-480-7251,양구군청 문화관광과,양구DMZ,1


# 4) 관광사진갤러리 API

4-1) galleryList1 : 관광사진갤러리 목록 조회

In [ ]:
# 관광사진갤러리 목록 조회 / arrange(A:촬영일, B:제목, C:수정일) parameter 사용 가능 / galSearchKeyword : 검색 키워드
service = "galleryList1"
url = f"http://apis.data.go.kr/B551011/PhotoGalleryService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=3&_type=json"

res = requests.get(url)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['galContentId', 'galContentTypeId', 'galTitle', 'galWebImageUrl',
       'galCreatedtime', 'galModifiedtime', 'galPhotographyMonth',
       'galPhotographyLocation', 'galPhotographer', 'galSearchKeyword'],
      dtype='object')


,galContentId,galContentTypeId,galTitle,galWebImageUrl,galCreatedtime,galModifiedtime,galPhotographyMonth,galPhotographyLocation,galPhotographer,galSearchKeyword
0,1002144,17,청설모,http://tong.visitkorea.or.kr/cms2/website/44/1...,20100420024817,20150818231341,201004,서울 경복궁,한국관광공사 김지호,"청설모, 동물"
1,1002619,17,경복궁 꽃담,http://tong.visitkorea.or.kr/cms2/website/19/1...,20100420222711,20150818233000,201004,서울,한국관광공사 김지호,"경복궁 자경전 꽃담 문양, 고궁"
2,1004457,17,국립고궁박물관,http://tong.visitkorea.or.kr/cms2/website/57/1...,20100421150936,20150818154827,201004,서울 경복궁,한국관광공사 김지호,국립고궁박물관


4-2) gallerySearchList1 : 관광갤러리 키워드검색 목록조회

In [ ]:
# 관광사진갤러리 키워드검색 목록 조회 / arrange(A:촬영일, B:제목, C:수정일) parameter 사용 가능 / keyword : 검색 키워드
service = "gallerySearchList1"
url = f"http://apis.data.go.kr/B551011/PhotoGalleryService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=3&_type=json"

params = {
    "keyword" : "동물"
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['galContentId', 'galContentTypeId', 'galTitle', 'galWebImageUrl',
       'galCreatedtime', 'galModifiedtime', 'galPhotographyMonth',
       'galPhotographyLocation', 'galPhotographer', 'galSearchKeyword'],
      dtype='object')


,galContentId,galContentTypeId,galTitle,galWebImageUrl,galCreatedtime,galModifiedtime,galPhotographyMonth,galPhotographyLocation,galPhotographer,galSearchKeyword
0,1002144,17,청설모,http://tong.visitkorea.or.kr/cms2/website/44/1...,20100420024817,20150818231341,201004,서울 경복궁,한국관광공사 김지호,"청설모, 동물"
1,1013256,17,조랑말,http://tong.visitkorea.or.kr/cms2/website/56/1...,20100503104900,20150819145307,201004,제주도,한국관광공사 김지호,"조랑말, 말, 동물"
2,1090582,17,다람쥐,http://tong.visitkorea.or.kr/cms2/website/82/1...,20100913134829,20150904101922,201008,강원도 태백,한국관광공사 김지호,"다람쥐,동물"


4-3) galleryDetailList1 : 관광사진갤러리 상세목록 조회

In [ ]:
# 관광사진갤러리 상세 목록 조회 / arrange(A:촬영일, B:제목, C:수정일) parameter 사용 가능 / title : 제목
service = "galleryDetailList1"
url = f"http://apis.data.go.kr/B551011/PhotoGalleryService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=3&_type=json"

params = {
    "title" : "청계천 야경"
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['galContentId', 'galContentTypeId', 'galTitle', 'galWebImageUrl',
       'galCreatedtime', 'galModifiedtime', 'galPhotographyMonth',
       'galPhotographyLocation', 'galPhotographer', 'galSearchKeyword'],
      dtype='object')


,galContentId,galContentTypeId,galTitle,galWebImageUrl,galCreatedtime,galModifiedtime,galPhotographyMonth,galPhotographyLocation,galPhotographer,galSearchKeyword
0,1199643,17,청계천 야경,http://tong.visitkorea.or.kr/cms2/website/43/1...,20110209150045,20151005091445,201012,서울,한국관광공사 이범수,"청계천 야경, 청계광장, 크리스마스트리, 연말연시, 서울야경"
1,1199663,17,청계천 야경,http://tong.visitkorea.or.kr/cms2/website/63/1...,20110209150358,20151005091509,201012,서울,한국관광공사 이범수,"청계천 야경, 연말연시, 트리, 서울야경"
2,1199668,17,청계천 야경,http://tong.visitkorea.or.kr/cms2/website/68/1...,20110209150533,20151005091623,201012,서울,한국관광공사 이범수,"청계천 야경, 연말연시, 트리, 서울야경"


4-4) gallerySyncDetailList1 : 관광사진정보 동기화 조회

In [ ]:
# 관광사진정보 동기화 조회 / galUseFlag(콘텐츠표출여부), galModifiedTime(콘텐츠변경일자) parameter 사용 가능
service = "gallerySyncDetailList1"
url = f"http://apis.data.go.kr/B551011/PhotoGalleryService1/{service}?serviceKey={key}&MobileApp=AppTest&MobileOS=ETC&pageNo=1&numOfRows=3&_type=json"

params = {
    "title" : "청계천 야경"
}

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['galContentId', 'galContentTypeId', 'galTitle', 'galWebImageUrl',
       'galCreatedtime', 'galModifiedtime', 'galPhotographyMonth',
       'galPhotographyLocation', 'galPhotographer', 'galSearchKeyword',
       'galUseFlag'],
      dtype='object')


,galContentId,galContentTypeId,galTitle,galWebImageUrl,galCreatedtime,galModifiedtime,galPhotographyMonth,galPhotographyLocation,galPhotographer,galSearchKeyword,galUseFlag
0,1199643,17,청계천 야경,http://tong.visitkorea.or.kr/cms2/website/43/1...,20110209150045,20151005091445,201012,서울,한국관광공사 이범수,"청계천 야경, 청계광장, 크리스마스트리, 연말연시, 서울야경",1
1,1199663,17,청계천 야경,http://tong.visitkorea.or.kr/cms2/website/63/1...,20110209150358,20151005091509,201012,서울,한국관광공사 이범수,"청계천 야경, 연말연시, 트리, 서울야경",1
2,1199668,17,청계천 야경,http://tong.visitkorea.or.kr/cms2/website/68/1...,20110209150533,20151005091623,201012,서울,한국관광공사 이범수,"청계천 야경, 연말연시, 트리, 서울야경",1


# 5) 공공데이터 Portal

5-1) 기상청 : 단기 예보 조회(구_동네예보) API

In [ ]:
# 초단기실황 조회 API / base_date(YYYYMMDD), base_time(정시단위, 매시각 40분 이후 호출 가능), nx, ny는 필수
# +999 이상, -999 이하 값은 결측치
# 단기예보 - BaseTime : 0200, 0500, 0800, 1100, 1400, 1700, 2000, 2300 (각 시간 10분 뒤부터 제공)
# >> POP(강수확률, %), PTY(강수형태), PCP(1시간 강수량, 1mm), REH(습도, %), SNO(1시간 신적설, 1cm), SKY(하늘상태), TMP(1시간 기온), TMN(일 최저기온), TMX(일 최고기온), UUU(동서 풍속), VVV(남북풍속), WAV(파고), VEC(풍향), WSD(풍속)
# 초단기실황 : T1H(기온), RN1(1시간 강수량), UUU, VVV, REH, PTY, VEC(풍향, deg), WSD(풍속, m/s)
# 초단기예보 : T1H, RN1, SKY, UUU, VVV, REH, PTY, LGH(낙뢰, KA), VEC, WSD
# >> 매시간 30분 생성, 10분마다 최신정보 업데이트(기온, 습도, 바람 등)

# 단기예보 = getVilageFcst
# 초단기예보 = getVilageFcst
# 초단기실황 = getUltraSrtNcst

service = "getUltraSrtNcst"
url = f"http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"

params = {
    "dataType":"json",
    "base_date":"20240606",
    "base_time":"1200",
    "nx":"55",
    "ny":"127"
    }

res = requests.get(url, params=params)
data = res.json()['response']['body']['items']['item']
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['baseDate', 'baseTime', 'category', 'nx', 'ny', 'obsrValue'], dtype='object')


,baseDate,baseTime,category,nx,ny,obsrValue
0,20240606,1200,PTY,55,127,0
1,20240606,1200,REH,55,127,57
2,20240606,1200,RN1,55,127,0
3,20240606,1200,T1H,55,127,26.6
4,20240606,1200,UUU,55,127,2.2


5-2) 부산테마여행정보 API

In [ ]:
# 부산테마여행 국문정보 / param - UC_SEQ : 콘텐츠 ID / response - ITEMCNTNTS=상세내용
service = "getRecommendedKr" # 영문정보 : getRecommendedEn , 일문정보 : getRecommendedJa, 중문간체정보 : getRecommendedZhs, 중문번체정보 : getRecommendedZht
url = f"http://apis.data.go.kr/6260000/RecommendedService/{service}"

params = {
    "resultType":"JSON"
    }

res = requests.get(url, params=params)
data = res.json()["getRecommendedKr"]["item"]
df = pd.DataFrame(data).head()
print(df.columns)
df

Index(['UC_SEQ', 'MAIN_TITLE', 'GUGUN_NM', 'CATE2_NM', 'LAT', 'LNG', 'PLACE',
       'TITLE', 'SUBTITLE', 'MAIN_PLACE', 'ADDR1', 'ADDR2', 'CNTCT_TEL',
       'HOMEPAGE_URL', 'TRFC_INFO', 'USAGE_DAY', 'HLDY_INFO',
       'USAGE_DAY_WEEK_AND_TIME', 'USAGE_AMOUNT', 'MIDDLE_SIZE_RM1',
       'MAIN_IMG_NORMAL', 'MAIN_IMG_THUMB', 'ITEMCNTNTS'],
      dtype='object')


,UC_SEQ,MAIN_TITLE,GUGUN_NM,CATE2_NM,LAT,LNG,PLACE,TITLE,SUBTITLE,MAIN_PLACE,...,HOMEPAGE_URL,TRFC_INFO,USAGE_DAY,HLDY_INFO,USAGE_DAY_WEEK_AND_TIME,USAGE_AMOUNT,MIDDLE_SIZE_RM1,MAIN_IMG_NORMAL,MAIN_IMG_THUMB,ITEMCNTNTS
0,58,초량이바구길,동구,도보여행,35.116350,129.03874,초량이바구길,이야기로 피어난 어제의 기억 초량이바구길,이바구 꽃이 피었습니다,,...,http://www.2bagu.co.kr,"도시철도 1호선 부산역 7번 출구 도보 2분, 초량역 1번 출구 도보 8분\n버스 ...",장소별 상이,장소별 상이,,장소별 상이,,https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,부산항을 기준으로 근처에 보이는 산 중턱마다 죄다 빽빽하게 들어서 있는 주택들.\n...
1,254,절영해안산책로,영도구,도보여행,35.080116,129.04251,절영해안산책로,"걷기만 해도 힐링, 절영해안산책로",바다를 벗 삼아 걷는 길,절영해안산책로,...,,"버스 508, 6, 7, 70, 71, 82, 85, 9 영도구1, 영도구5 부산보...",,,,,휠체어 접근 가능 구간 있음(절영해안산책로 입구 ~ 흰여울해안터널 / 절영해랑길),https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,부산하면 떠오르는 바다! 해수욕장은 많이 가봤다고? 색다른 부산바다의 매력을 느끼고...
2,282,스카이워크 시리즈,남구,이색여행,35.100853,129.12430,스카이워크 시리즈,"부산 3대 스카이워크 오륙도, 송도, 청사포","바다 위를 걷는 즐거움, 부산 스카이워크를 가다",스카이워크 시리즈,...,,"오륙도 스카이워크 \n도시철도 2호선 경성대·부경대역 5번 출구 → 131, 24,...",,"연중무휴(눈, 비, 강풍 및 시설 개·보수 시 개방 제한)",오륙도 스카이워크 \n09:00~18:00(입장마감 17:50)\n송도 구름산책로\...,무료,"오륙도 스카이워크\n장애인 주차구역, 장애인 화장실, 휠체어접근 가능(스카이워크 입...",https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,백사장과 해안도로를 넘나들며 만나는 부산 바다가 익숙한 당신에게 조금 특별한 바다전...
3,283,달맞이길/문탠로드,해운대구,도보여행,35.156742,129.18070,달맞이길/문탠로드,일출과 월출 모두를 품은 달맞이길 & 문탠로드,걷기 좋은 도심 속 숲길,,...,,"도시철도 2호선 해운대역 1번 출구 → 해운대구2, 해운대구10 마을버스 환승\n버...",,,,무료,"장애인 화장실, 장애인 주차구역, 휠체어 접근 가능",https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,해운대 삼포길의 시작점이자 갈맷길 1코스와 2코스를 잇는 길.\n월출이 아름다운 부...
4,284,동백해안산책로,해운대구,도보여행,35.151962,129.15263,동백해안산책로,"바다와 산, 그리고 부산을 걷다",부산을 담은 동백해안산책로,,...,,"도시철도 2호선 동백역 1번 출구 도보 20분\n버스 139, 307, 1003, ...",,연중무휴,상시\n누리마루 APEC하우스 개방 09:00~18:00,무료,"장애인 화장실, 장애인 주차구역, 휠체어 접근 가능, 누리마루 APEC하우스 내 휠...",https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,해운대해수욕장에서 탁 트인 바다를 바라보며 사색에 잠겨 걷다 보면 백사장 끝자락에 ...


5-3) 부산관광공사 : 부산 음식테마거리

In [ ]:
https://www.data.go.kr/data/15096646/openapi.do

5-4) 부산버스정보시스템

In [ ]:
부산버스정보시스템

5-5) 부산광역시_연극 목록, 부산광역시_뮤지컬 목록 서비스, 부산광역시_콘서트 목록 서비스, 부산광역시_전시 목록 서비스

5-6) 부산광역시_부산맛집정보 서비스

5-7) 부산광역시_종합병원 현황

5-8) 부산광역시_공영주차장 정보 조회

5-9) 산광역시_부산쇼핑정보 서비스

5-10) 부산광역시_해수욕장 수질 정보